In [33]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

## Program to investigate concepts of tangent linear models, adjoint sensitivity, and singular vectors with the Lorenz 63 model.

Developed by Greg Hakim, Ryan Torn, Aneesh Subramanian

###  Compute right-hand side of the Lorenz 63 model equations

In [34]:
def lorenz(x, y, z, s=10, r=28, b=2.667):
    dxdt = s*(y - x)
    dydt = r*x - y - x*z
    dzdt = x*y - b*z
    return dxdt, dydt, dzdt

###  Compute the tangent linear model matrix for Lorenz 63 model by brute force based on input forecast trajectory

In [35]:
def lorenz_tlm(tvec, x, y, z, s=10, r=28, b=2.667):

    v1 = np.array([1, 0, 0])
    v2 = np.array([0, 1, 0])
    v3 = np.array([0, 0, 1])

    for t in range(len(tvec)-1):

      tlm = np.array([[-s, s, 0], [(r-z[t]), -1., -x[t]], [y[t], x[t], -b]])

      v1 = v1 + np.matmul(tlm,v1)*(tvec[t+1]-tvec[t])
      v2 = v2 + np.matmul(tlm,v2)*(tvec[t+1]-tvec[t])
      v3 = v3 + np.matmul(tlm,v3)*(tvec[t+1]-tvec[t])

    M = np.transpose([v1, v2, v3])
    
    return M

###  Compute the distance between two points in Lorenz 63 space based on Euclidian distance norm

In [36]:
def euclid_norm(x1, y1, z1, x2, y2, z2):

    print("x, y, z error: ",x2-x1,y2-y1,z2-z1)

    err = np.sqrt((x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2)
    return err


dt         = 0.002   #  time step of the model (do not need to change)
num_steps  = 20000   #  Number of time steps for full model trajectory
 
step1      = 1000    #  model time step to initiate forecast from
fcst_len   = 1.0     #  forecast length in model timesteps
tl_freq    = 1       #  frequency of model timesteps to use in computing tangent linear model
sing_val   = 1       #  singular value plot/compute
ic_error   = np.array([0.1, 0.1, 0.1])  # initial condition error to add to forecast
xf_sens    = np.array([0., 0., 1.])  #  metric to compute sensitivity in x, y, z space (0, 0, 1) means compute sensitivity of z forecast

plot3d     = True    #  True to plot forecast in 3D space
plotxz     = True    #  True to plot in x-z plane
plot_nlfcst = False  #  True to plt non-linear forecast
plot_sv     = True   #  True to plot singular vectors
sv_plot_freq = 20    #  Frequency of time steps to plot singular vectors

fcst_steps = int(fcst_len / dt)

###  Create arrays needed for calculations.  Need one more for the initial values

In [37]:
xtraj = np.empty(num_steps + 1)
ytraj = np.empty(num_steps + 1)
ztraj = np.empty(num_steps + 1)

tfcst = np.empty(fcst_steps + 1)
xfcst = np.empty(fcst_steps + 1)
yfcst = np.empty(fcst_steps + 1)
zfcst = np.empty(fcst_steps + 1)

xtrue = np.empty(fcst_steps + 1)
ytrue = np.empty(fcst_steps + 1)
ztrue = np.empty(fcst_steps + 1)

ttlm  = []
xtlm  = []
ytlm  = []
ztlm  = []

# Set initial values
xtraj[0], ytraj[0], ztraj[0] = (0., 1., 1.05)

### Step through time, calculating the partial derivatives at the current point and using them to estimate the next point

In [38]:
for i in range(num_steps):
   x_dot, y_dot, z_dot = lorenz(xtraj[i], ytraj[i], ztraj[i])
   xtraj[i + 1] = xtraj[i] + (x_dot * dt)
   ytraj[i + 1] = ytraj[i] + (y_dot * dt)
   ztraj[i + 1] = ztraj[i] + (z_dot * dt)

###  Now initiate forecast from specified time step, add initial condition error

In [39]:
tfcst[0] = 0.0
xtrue[0], ytrue[0], ztrue[0] = (xtraj[step1], ytraj[step1], ztraj[step1])
xfcst[0], yfcst[0], zfcst[0] = (xtrue[0]+ic_error[0], ytrue[0]+ic_error[1], ztrue[0]+ic_error[2])

###  Step the forecast and truth through time

In [40]:
for i in range(fcst_steps):

   tfcst[i+1] = tfcst[i] + dt

   x_dot, y_dot, z_dot = lorenz(xfcst[i], yfcst[i], zfcst[i])
   xfcst[i + 1] = xfcst[i] + (x_dot * dt)
   yfcst[i + 1] = yfcst[i] + (y_dot * dt)
   zfcst[i + 1] = zfcst[i] + (z_dot * dt)

   x_dot, y_dot, z_dot = lorenz(xtrue[i], ytrue[i], ztrue[i])
   xtrue[i + 1] = xtrue[i] + (x_dot * dt)
   ytrue[i + 1] = ytrue[i] + (y_dot * dt)
   ztrue[i + 1] = ztrue[i] + (z_dot * dt)

   #  Add this forecast time to tangent linear trajectory, if needed
   if ( i % tl_freq == 0 ):

      ttlm.append(tfcst[i])
      xtlm.append(xtrue[i])
      ytlm.append(ytrue[i])
      ztlm.append(ztrue[i])

###  Compute tangent linear model and estimated forecast error

In [41]:
M = lorenz_tlm(ttlm, xtlm, ytlm, ztlm)
tl_xf = np.matmul(M,ic_error)

print("tangent linear estimated error: ", euclid_norm(tl_xf[0], tl_xf[1], tl_xf[2], 0., 0., 0.))
print("actual Non-linear model error: ", euclid_norm(xfcst[-1], yfcst[-1], zfcst[-1], xtrue[-1], ytrue[-1], ztrue[-1]))

#  Compute initial condition sensitivity from adjoint of tangent linear model
xi_sens = np.matmul(np.transpose(M),xf_sens)

#  add initial condition error to truth consistent with sensitivity, but same size as ic_error
xi_sens[:] = xi_sens[:] / np.sqrt(np.sum(xi_sens[:]**2)) * np.sqrt(np.sum(ic_error[:]**2))

xsens = np.empty(fcst_steps + 1)
ysens = np.empty(fcst_steps + 1)
zsens = np.empty(fcst_steps + 1)
xsens[0], ysens[0], zsens[0] = (xtrue[0]+xi_sens[0], ytrue[0]+xi_sens[1], ztrue[0]+xi_sens[2])

x, y, z error:  0.11648942538284433 0.15712951517251955 -0.08661675100867663
tangent linear estimated error:  0.21392038780741632
x, y, z error:  0.11682624268418174 0.15680152346669463 -0.08828333209996231
actual Non-linear model error:  0.21454378450062808


###  Step the sensitivity IC error forecast forward in time, compute error

In [42]:
for i in range(fcst_steps):

   x_dot, y_dot, z_dot = lorenz(xsens[i], ysens[i], zsens[i])
   xsens[i + 1] = xsens[i] + (x_dot * dt)
   ysens[i + 1] = ysens[i] + (y_dot * dt)
   zsens[i + 1] = zsens[i] + (z_dot * dt)

print("Error from forecast with sensitivity initial condition error: ", 
        euclid_norm(xsens[-1], ysens[-1], zsens[-1], xtrue[-1], ytrue[-1], ztrue[-1]))

#  Compute singular value decomposition of tangent linear model
u, s, vt = np.linalg.svd(M, full_matrices=True)

print("Singular Value: ",s[sing_val-1])

#  Add initial-time singular vector to forecast
ic_sing = np.array(vt[sing_val-1,:])
ic_sing[:] = ic_sing[:] / np.sqrt(np.sum(ic_sing[:]**2)) * np.sqrt(np.sum(ic_error[:]**2))

xsing = np.empty(fcst_steps + 1)
ysing = np.empty(fcst_steps + 1)
zsing = np.empty(fcst_steps + 1)
xsing[0], ysing[0], zsing[0] = (xtrue[0]+ic_sing[0], ytrue[0]+ic_sing[1], ztrue[0]+ic_sing[2])

#  Step forecast with singular vector IC error forward in time, compute error
for i in range(fcst_steps):

   x_dot, y_dot, z_dot = lorenz(xsing[i], ysing[i], zsing[i])
   xsing[i + 1] = xsing[i] + (x_dot * dt)
   ysing[i + 1] = ysing[i] + (y_dot * dt)
   zsing[i + 1] = zsing[i] + (z_dot * dt)

print("Error from forecast with SV initial condition error: ",
        euclid_norm(xsing[-1], ysing[-1], zsing[-1], xtrue[-1], ytrue[-1], ztrue[-1]) )

if ( plot3d ):

  fig = plt.figure()
  ax = fig.add_subplot(projection = '3d')

  ax.plot(xtraj, ytraj, ztraj, lw=0.5, color='lightgray')
  ax.plot(xtrue, ytrue, ztrue, lw=0.5, color='red')
  ax.set_xlabel("X Axis")
  ax.set_ylabel("Y Axis")
  ax.set_zlabel("Z Axis")

  plt.savefig('lorenz_xyz.png',format='png',dpi=150,bbox_inches='tight')
  plt.close(fig)


if ( plotxz ):

  fig = plt.figure()
  ax = fig.gca()
  ax.plot(xtraj, ztraj, lw=0.5, color='lightgray')
  ax.plot(xtrue, ztrue, '-', color='red')
  ax.plot(xtrue[0], ztrue[0], 'o', color='red')
  ax.plot(xtrue[-1], ztrue[-1], '^', color='red')
  
  if ( plot_nlfcst ):
    ax.plot(xfcst, zfcst, '-', color='green')
    ax.plot(xfcst[0], zfcst[0], 'o', color='green')
    ax.plot(xfcst[-1], zfcst[-1], '^', color='green')

  xmin = np.min([np.min(xtrue), np.min(xfcst)])
  xmax = np.max([np.max(xtrue), np.max(xfcst)])
  zmin = np.min([np.min(ztrue), np.min(zfcst)])
  zmax = np.max([np.max(ztrue), np.max(zfcst)])

  if ( plot_sv ):
    ax.plot([xtrue[0], xsing[0]], [ztrue[0], zsing[0]], '-', color='blue')
    ax.plot([xtrue[-1], xsing[-1]], [ztrue[-1], zsing[-1]], '-', color='blue')
    for i in range(0,len(xsing),sv_plot_freq):
      ax.plot([xtrue[i], xsing[i]], [ztrue[i], zsing[i]], '-', color='blue')

  plt.axis([xmin-(xmax-xmin)*0.08, xmax+(xmax-xmin)*0.08, zmin-(zmax-zmin)*0.08, zmax+(zmax-zmin)*0.08])
  plt.xlabel('x')
  plt.ylabel('z')

  plt.savefig('lorenz_xz.png',format='png',dpi=150,bbox_inches='tight')
  plt.close(fig)

x, y, z error:  0.10133397875501338 0.03140522363315945 -0.20555588153571946
Error from forecast with sensitivity initial condition error:  0.23131814402613773
Singular Value:  1.4849703940755725
x, y, z error:  0.14047047732991835 0.14189110868375288 -0.1639058509980238
Error from forecast with SV initial condition error:  0.2583218336033291
